# Sequential Orchestration

In the sequential pattern, agents are organized in a pipeline. Each agent processes the task in turn, passing its output to the next agent in the sequence. This is ideal for workflows where each step builds upon the previous one, such as document review, data processing pipelines, or multi-stage reasoning.

* [Agent-orchestration Guide - Sequential](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-orchestration/sequential?pivots=programming-language-python)
* [Original Code](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/getting_started_with_agents/multi_agent_orchestration/step2_sequential.py)

The following sample demonstrates how to create a sequential orchestration for executing multiple agents in sequence, i.e. the output of one agent is the input to the next agent.

This sample demonstrates the basic steps of creating and starting a runtime, creating a sequential orchestration, invoking the orchestration, and finally waiting for the results.

In [1]:
# Libraries
from dotenv import load_dotenv
import os
import asyncio

from semantic_kernel.agents import Agent, ChatCompletionAgent, SequentialOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent

In [3]:
#load variables
load_dotenv()

# Variables - Azure Services
azopenai_ep=os.environ["AZURE_OPENAI_ACCOUNT"]
azopenai_key=os.environ["AZURE_OPENAI_KEY"]
azopenai_model=os.environ["AZURE_OPENAI_MODEL"]


In [4]:
# Azure Chat Completion Client
az_chat_completion = AzureChatCompletion(
    deployment_name=azopenai_model,
    api_key=azopenai_key,
    endpoint=azopenai_ep 
)

### Create the agents

In [5]:
def get_agents() -> list[Agent]:
    """Return a list of agents that will participate in the sequential orchestration.

    Feel free to add or remove agents.
    """
    concept_extractor_agent = ChatCompletionAgent(
        name="ConceptExtractorAgent",
        instructions=(
            "You are a marketing analyst. Given a product description, identify:\n"
            "- Key features\n"
            "- Target audience\n"
            "- Unique selling points\n\n"
        ),
        service=AzureChatCompletion(),
    )
    writer_agent = ChatCompletionAgent(
        name="WriterAgent",
        instructions=(
            "You are a marketing copywriter. Given a block of text describing features, audience, and USPs, "
            "compose a compelling marketing copy (like a newsletter section) that highlights these points. "
            "Output should be short (around 150 words), output just the copy as a single text block."
        ),
        service=AzureChatCompletion(),
    )
    format_proof_agent = ChatCompletionAgent(
        name="FormatProofAgent",
        instructions=(
            "You are an editor. Given the draft copy, correct grammar, improve clarity, ensure consistent tone, "
            "give format and make it polished. Output the final improved copy as a single text block."
        ),
        service=AzureChatCompletion(),
    )

    # The order of the agents in the list will be the order in which they are executed
    return [concept_extractor_agent, writer_agent, format_proof_agent]


### Optional: Observe Agent Responses

You can define a callback to observe and print the output from each agent as the sequence progresses.

In [6]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents."""
    print(f"# {message.name}\n{message.content}")

### Create main logic

In [7]:
# 1. Create a sequential orchestration with multiple agents and an agent
#    response callback to observe the output from each agent.
agents = get_agents()
sequential_orchestration = SequentialOrchestration(
    members=agents,
    agent_response_callback=agent_response_callback,
)

# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await sequential_orchestration.invoke(
    task="An eco-friendly stainless steel water bottle that keeps drinks cold for 24 hours",
    runtime=runtime,
)

# ConceptExtractorAgent
- **Key Features:**
  - Eco-friendly and reusable
  - Made from durable stainless steel
  - Keeps drinks cold for up to 24 hours
  - Insulated design

- **Target Audience:**
  - Environmentally conscious consumers
  - Outdoor enthusiasts (e.g., hikers, campers)
  - Fitness enthusiasts and gym-goers
  - Office workers or students who need cold beverages throughout the day

- **Unique Selling Points:**
  - Sustainable alternative to single-use plastic bottles
  - Long-lasting cold beverage retention (24 hours)
  - Built with high-quality, durable stainless steel
  - Appeals to eco-conscious buyers looking to reduce their environmental footprint
# WriterAgent
Say goodbye to single-use plastics and hello to the ultimate hydration hero! Our eco-friendly stainless steel bottles are designed with your lifestyle—and the planet—in mind. Whether you're trekking through the wilderness, hitting the gym, or powering through a workday, this insulated bottle keeps your drinks 

In [8]:
# 4. Wait for the results
value = await orchestration_result.get(timeout=20)
print(f"***** Final Result *****\n{value}\n")

# 5. Stop the runtime when idle
await runtime.stop_when_idle()

***** Final Result *****
Say goodbye to single-use plastics and embrace the ultimate hydration solution! Our eco-friendly stainless steel bottles are thoughtfully designed to fit seamlessly into your lifestyle while protecting the planet. Whether you're exploring the great outdoors, powering through a workout, or staying focused at the office, this insulated bottle ensures your drinks remain refreshingly cold for up to 24 hours. Built with durability and sustainability in mind, it’s perfect for any adventure while helping reduce waste. Make the switch today—choose reusable and stay hydrated, one sip at a time!

